<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Face%20Age%20RegressionTorch/AgePredictionTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

 93% 309M/331M [00:12<00:00, 24.4MB/s]
100% 331M/331M [00:12<00:00, 27.7MB/s]


In [3]:
!unzip -qq utkface-new.zip

In [4]:
import os
import numpy as np
import pandas as pd
import cv2

In [5]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm

In [7]:
images=[]
labels=[]
for image_name in os.listdir('crop_part1')[0:9000]:
  parts=image_name.split('_')
  labels.append(int(parts[0]))
  img=cv2.imread(f'crop_part1/{image_name}')
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  images.append(img)

In [8]:
print(images[0].shape)

(200, 200, 3)


In [9]:
images=pd.Series(images,name='images')
ages=pd.Series(labels,name='ages')
df=pd.concat([images,ages],axis=1)
df.head()

,images,ages
0,"[[[51, 41, 40], [51, 41, 40], [50, 40, 39], [4...",23
1,"[[[54, 38, 22], [53, 37, 21], [48, 32, 16], [4...",37
2,"[[[50, 33, 26], [52, 35, 28], [56, 37, 31], [5...",1
3,"[[[11, 10, 8], [10, 9, 7], [12, 7, 4], [13, 5,...",3
4,"[[[207, 170, 144], [207, 170, 144], [208, 171,...",1


In [10]:
under_4=[]
for i in range(len(df)):
  if df['ages'].iloc[i]<=4:
    under_4.append(df.iloc[i])

under_4=pd.DataFrame(under_4)
under_4=under_4.sample(frac=0.1)

up_4=df[df['ages']>4]

df=pd.concat([under_4,up_4])

df=df[df['ages']<80]

In [11]:
!mkdir new

In [12]:
from numpy.ma.core import append
df=pd.DataFrame(df)
new_df=pd.DataFrame(columns=['image','age'])
for i in range(len(df)):
  new_df.loc[i]={'image':f'{i}.jpg','age':df['ages'].iloc[i]}
  df['images'].iloc[i]=cv2.resize(df['images'].iloc[i],(160,160))
  cv2.imwrite(f'/content/new/{i}.jpg',df['images'].iloc[i])



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# new_df.head(50)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [13]:
from torchvision.transforms.transforms import Resize
from torch.utils.data import Dataset
from PIL import Image

class PathologyPlantsDataset(Dataset):
  """
  The Class will act as the container for our dataset. It will take your dataframe, the root path, and also the transform function for transforming the dataset.
  """
  def __init__(self, data_frame, root_dir, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform
    
  def __len__(self):
        # Return the length of the dataset
        return len(self.data_frame)
    
  def __getitem__(self, idx):
        # Return the observation based on an index. Ex. dataset[0] will return the first element from the dataset, in this case the image and the label.
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, -1]
        
        if self.transform:
            image = self.transform(image)
    
        return image, label


transform = transforms.Compose([
    # transforms.PILToTensor(),
    transforms.Resize((160,160)),
    # transforms.RandomRotation(10),
    transforms.ToTensor(),
    # torchvision.transforms.Normalize((0), (1))
])


dataset= PathologyPlantsDataset(data_frame=new_df,root_dir='/content/new', transform=transform)

In [14]:
train_data_size=int(len(dataset)*0.9)
train_set, val_set = torch.utils.data.random_split(dataset, [train_data_size, len(dataset)-train_data_size])

In [15]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,160,(3,3),(1,1),(0,0))
    self.conv2=nn.Conv2d(160,64,(3,3),(1,1),(0,0))
    self.conv3=nn.Conv2d(64,32,(3,3),(1,1),(0,0))
    self.conv4=nn.Conv2d(1024,2048,(3,3),(1,1),(0,0))
    self.conv5=nn.Conv2d(2048,1024,(3,3),(1,1),(1,1))
    self.conv6=nn.Conv2d(1024,128,(3,3),(1,1),(1,1))

    self.fc1=nn.Linear(32*18*18,256)
    self.fc2=nn.Linear(256,64)
    self.fc3=nn.Linear(64,16)
    self.fc4=nn.Linear(16,8)
    self.fc5=nn.Linear(8,1)

    self.norm1=nn.BatchNorm2d(160)
    self.norm2=nn.BatchNorm2d(64)
    self.norm3=nn.BatchNorm2d(32)
  
  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=self.norm1(x)
    x=F.avg_pool2d(x,kernel_size=(2,2))
  
    x=F.relu(self.conv2(x))
    x=self.norm2(x)
    x=F.avg_pool2d(x,kernel_size=(2,2))

    x=F.relu(self.conv3(x))
    x=self.norm3(x)
    x=F.avg_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv4(x))
    # x=F.avg_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv5(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv6(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    # x=F.relu(self.conv7(x))
    # x=F.max_pool2d(x,kernel_size=(2,2))

    x=torch.flatten(x,start_dim=1)

    x=F.relu(self.fc1(x))
    x=F.sigmoid(self.fc2(x))
    x=F.relu(self.fc3(x))
    x=F.relu(self.fc4(x))
    x=self.fc5(x)
    return x

In [16]:
model=Model().to(device)

In [17]:
#hyper parameters
batch_size=64
epochs=5
lr=0.001

In [18]:
train_data_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=batch_size)

In [19]:
#compile 
optimizer=torch.optim.Adam(model.parameters(),lr=lr)
loss_function=nn.L1Loss()

In [20]:
model.train()
torch.cuda.empty_cache()
for epoch in range(epochs):

  train_loss=0.0
  for images,labels in tqdm(train_data_loader):
    images,labels=images.to(device),labels.to(device)
    optimizer.zero_grad()

    preds=model(images)
    ls=loss_function(preds,labels.float())
    ls.backward()
    optimizer.step()
    train_loss+=ls
  
  total_loss=train_loss/len(train_data_loader)
  print(f"epochs: {epoch} , loss: {total_loss}")

  0%|          | 0/97 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
 99%|█████████▉| 96/97 [00:38<00:00,  3.15it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
100%|

epochs: 0 , loss: 32.905494689941406


100%|██████████| 97/97 [00:30<00:00,  3.21it/s]


epochs: 1 , loss: 25.871782302856445


100%|██████████| 97/97 [00:30<00:00,  3.17it/s]


epochs: 2 , loss: 17.2052001953125


100%|██████████| 97/97 [00:30<00:00,  3.19it/s]


epochs: 3 , loss: 17.151866912841797


100%|██████████| 97/97 [00:30<00:00,  3.18it/s]

epochs: 4 , loss: 17.147132873535156


In [21]:
torch.save(model.state_dict(),"face18.pth")